# RDD = resilient distributed dataset

In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

26/01/26 08:59:08 WARN Utils: Your hostname, codespaces-f3c854 resolves to a loopback address: 127.0.0.1; using 10.0.0.133 instead (on interface eth0)
26/01/26 08:59:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


26/01/26 08:59:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df_green = spark.read.parquet('data/pq/green/*/*')

```
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
```

In [4]:
rdd = df_green \
    .select('lpep_pickup_datetime', 'PULocationID', 'total_amount') \
    .rdd

In [9]:
# rdd of the Spark df
rdd

MapPartitionsRDD[13] at javaToPython at NativeMethodAccessorImpl.java:0

In [6]:
rdd.__dict__

{'_jrdd': JavaObject id=o43,
 'is_cached': False,
 'is_checkpointed': False,
 'has_resource_profile': False,
 'ctx': <SparkContext master=local[*] appName=test>,
 '_jrdd_deserializer': BatchedSerializer(CloudPickleSerializer(), -1),
 '_id': 13,
 'partitioner': None}

# First 10 rows

In [8]:
rdd.take(10)

Traceback (most recent call last):
  File "/workspaces/data-engineering-zoomcamp-homework/05-batch/spark/spark-3.3.2-bin-hadoop3/python/pyspark/serializers.py", line 458, in dumps
    return cloudpickle.dumps(obj, pickle_protocol)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/workspaces/data-engineering-zoomcamp-homework/05-batch/spark/spark-3.3.2-bin-hadoop3/python/pyspark/cloudpickle/cloudpickle_fast.py", line 73, in dumps
    cp.dump(obj)
  File "/workspaces/data-engineering-zoomcamp-homework/05-batch/spark/spark-3.3.2-bin-hadoop3/python/pyspark/cloudpickle/cloudpickle_fast.py", line 602, in dump
    return Pickler.dump(self, obj)
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "/workspaces/data-engineering-zoomcamp-homework/05-batch/spark/spark-3.3.2-bin-hadoop3/python/pyspark/cloudpickle/cloudpickle_fast.py", line 692, in reducer_override
    return self._function_reduce(obj)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/workspaces/data-engineering-zoomcamp-homework/05-ba

PicklingError: Could not serialize object: IndexError: tuple index out of range

In [ ]:
# compatibility issue between Python and PySpark versions, particularly when using Python 3.11 or newer with Spark 3.3.2.
# This error happens during the serialization process when Spark tries to convert your DataFrame to an RDD and then execute the take(10) operation. 
# The root cause is often related to how PySpark's cloudpickle library handles object serialization in certain Python versions.

In [12]:
# approach 1 - show first 10 rows
df_green.select('lpep_pickup_datetime', 'PULocationID', 'total_amount').show(10)

+--------------------+------------+------------+
|lpep_pickup_datetime|PULocationID|total_amount|
+--------------------+------------+------------+
| 2020-01-23 13:10:15|          74|       44.97|
| 2020-01-20 15:09:00|          67|       33.45|
| 2020-01-15 20:23:41|         260|         8.3|
| 2020-01-05 16:32:26|          82|         8.3|
| 2020-01-29 19:22:42|         166|       12.74|
| 2020-01-15 11:07:42|         179|         5.8|
| 2020-01-16 08:22:29|          41|       25.05|
| 2020-01-28 17:05:28|          75|       21.33|
| 2020-01-22 14:51:37|         152|         7.8|
| 2020-01-31 10:25:04|          75|       26.05|
+--------------------+------------+------------+
only showing top 10 rows



In [15]:
# returns a list -> no need .show()
# approach 2 - return a list of the first 10 Row objects
rdd.collect()[:10]

[Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 23, 13, 10, 15), PULocationID=74, total_amount=44.97),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 20, 15, 9), PULocationID=67, total_amount=33.45),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 15, 20, 23, 41), PULocationID=260, total_amount=8.3),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 5, 16, 32, 26), PULocationID=82, total_amount=8.3),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 29, 19, 22, 42), PULocationID=166, total_amount=12.74),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 15, 11, 7, 42), PULocationID=179, total_amount=5.8),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 16, 8, 22, 29), PULocationID=41, total_amount=25.05),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 28, 17, 5, 28), PULocationID=75, total_amount=21.33),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 22, 14, 51, 37), PULocationID=152, total_amount=7.8),
 Row(lpep_pickup_datetime=datetime.dateti

In [16]:
# original
# rows = rdd.take(10)

# workaround
rows = rdd.collect()[:10]
row = rows[0]

In [17]:
row

Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 23, 13, 10, 15), PULocationID=74, total_amount=44.97)

# Filter on datetime field

In [33]:
sc = spark.sparkContext  # This gets the SparkContext from SparkSession
sc

<SparkContext master=local[*] appName=test>

In [10]:
from datetime import datetime

In [34]:
# round1
# start = datetime(year=2020, month=1, day=1)

# def filter_outliers(row):
#     return row.lpep_pickup_datetime >= start


# round2
# import pickle
# # Define the start date inside the function to avoid closure issues
# def filter_outliers(row):
#     start = datetime(year=2020, month=1, day=1)
#     # Handle different row formats - try both attribute and index access
#     try:
#         pickup_time = row.lpep_pickup_datetime
#     except AttributeError:
#         try:
#             # If row is a Row object, access by field name
#             pickup_time = row['lpep_pickup_datetime']
#         except (TypeError, KeyError):
#             # If row is a tuple, access by index
#             pickup_time = row[0]  # Adjust index based on your data structure
    
#     return pickup_time >= start

# round 3
from datetime import datetime
from pyspark import Broadcast

# Create broadcast variable for the start date
start_date = sc.broadcast(datetime(year=2020, month=1, day=1))

def filter_outliers(row):
    # Access the broadcast variable
    start = start_date.value
    
    # Handle different row types safely
    if hasattr(row, 'lpep_pickup_datetime'):
        pickup_time = row.lpep_pickup_datetime
    elif isinstance(row, tuple) and len(row) >= 1:
        # If it's a tuple, assume first element is lpep_pickup_datetime
        pickup_time = row[0]
    else:
        return False
    
    return pickup_time >= start

In [31]:
# round 2
# Try to get the first element
try:
    result = rdd.filter(filter_outliers).first()
    print(result)
except Exception as e:
    print(f"Error: {e}")
    
    # Alternative: Try to see the structure of your data first
    print("Sample data structure:")
    sample = rdd.take(1)[0]
    print(type(sample))
    print(sample)

Error: Could not serialize object: IndexError: tuple index out of range
Sample data structure:


Traceback (most recent call last):
  File "/workspaces/data-engineering-zoomcamp-homework/05-batch/spark/spark-3.3.2-bin-hadoop3/python/pyspark/serializers.py", line 458, in dumps
    return cloudpickle.dumps(obj, pickle_protocol)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/workspaces/data-engineering-zoomcamp-homework/05-batch/spark/spark-3.3.2-bin-hadoop3/python/pyspark/cloudpickle/cloudpickle_fast.py", line 73, in dumps
    cp.dump(obj)
  File "/workspaces/data-engineering-zoomcamp-homework/05-batch/spark/spark-3.3.2-bin-hadoop3/python/pyspark/cloudpickle/cloudpickle_fast.py", line 602, in dump
    return Pickler.dump(self, obj)
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "/workspaces/data-engineering-zoomcamp-homework/05-batch/spark/spark-3.3.2-bin-hadoop3/python/pyspark/cloudpickle/cloudpickle_fast.py", line 692, in reducer_override
    return self._function_reduce(obj)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/workspaces/data-engineering-zoomcamp-homework/05-ba

PicklingError: Could not serialize object: IndexError: tuple index out of range

In [35]:
# round3
result = rdd.filter(filter_outliers).first()
print(result)

Traceback (most recent call last):
  File "/workspaces/data-engineering-zoomcamp-homework/05-batch/spark/spark-3.3.2-bin-hadoop3/python/pyspark/serializers.py", line 458, in dumps
    return cloudpickle.dumps(obj, pickle_protocol)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/workspaces/data-engineering-zoomcamp-homework/05-batch/spark/spark-3.3.2-bin-hadoop3/python/pyspark/cloudpickle/cloudpickle_fast.py", line 73, in dumps
    cp.dump(obj)
  File "/workspaces/data-engineering-zoomcamp-homework/05-batch/spark/spark-3.3.2-bin-hadoop3/python/pyspark/cloudpickle/cloudpickle_fast.py", line 602, in dump
    return Pickler.dump(self, obj)
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "/workspaces/data-engineering-zoomcamp-homework/05-batch/spark/spark-3.3.2-bin-hadoop3/python/pyspark/cloudpickle/cloudpickle_fast.py", line 692, in reducer_override
    return self._function_reduce(obj)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/workspaces/data-engineering-zoomcamp-homework/05-ba

PicklingError: Could not serialize object: IndexError: tuple index out of range

In [36]:
# round 4
from datetime import datetime

def filter_outliers(row):
    # Define start date inside the function to avoid closure capture
    start = datetime(year=2020, month=1, day=1)
    
    # Handle different row structures
    if hasattr(row, 'lpep_pickup_datetime'):
        pickup_time = row.lpep_pickup_datetime
    elif hasattr(row, '__getitem__') and len(row) > 0:
        # Try to access by index if it's a tuple-like structure
        try:
            pickup_time = row[0]  # lpep_pickup_datetime is first column
        except (TypeError, IndexError):
            return False
    else:
        return False
    
    return pickup_time >= start

result = rdd.filter(filter_outliers).first()
print(result)

Traceback (most recent call last):
  File "/workspaces/data-engineering-zoomcamp-homework/05-batch/spark/spark-3.3.2-bin-hadoop3/python/pyspark/serializers.py", line 458, in dumps
    return cloudpickle.dumps(obj, pickle_protocol)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/workspaces/data-engineering-zoomcamp-homework/05-batch/spark/spark-3.3.2-bin-hadoop3/python/pyspark/cloudpickle/cloudpickle_fast.py", line 73, in dumps
    cp.dump(obj)
  File "/workspaces/data-engineering-zoomcamp-homework/05-batch/spark/spark-3.3.2-bin-hadoop3/python/pyspark/cloudpickle/cloudpickle_fast.py", line 602, in dump
    return Pickler.dump(self, obj)
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "/workspaces/data-engineering-zoomcamp-homework/05-batch/spark/spark-3.3.2-bin-hadoop3/python/pyspark/cloudpickle/cloudpickle_fast.py", line 692, in reducer_override
    return self._function_reduce(obj)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/workspaces/data-engineering-zoomcamp-homework/05-ba

PicklingError: Could not serialize object: IndexError: tuple index out of range

# Group
* map - Create k-v pair of (k=(hour, zone), v=(amount, count)) for `each row`
* reduceByKey - combine amount and count `per (hour, zone) key`
* map - unwrap / unpack into (hour, zone, total_amt, total_count) per `aggregated row`

In [18]:
def prepare_for_grouping(row): 
    hour = row.lpep_pickup_datetime.replace(minute=0, second=0, microsecond=0)
    zone = row.PULocationID
    key = (hour, zone)
    
    amount = row.total_amount
    count = 1
    value = (amount, count)

    return (key, value)

In [19]:
def calculate_revenue(left_value, right_value):
    left_amount, left_count = left_value
    right_amount, right_count = right_value
    
    output_amount = left_amount + right_amount
    output_count = left_count + right_count
    
    return (output_amount, output_count)

In [20]:
from collections import namedtuple

In [21]:
RevenueRow = namedtuple('RevenueRow', ['hour', 'zone', 'revenue', 'count'])

In [22]:
def unwrap(row):
    return RevenueRow(
        hour=row[0][0], 
        zone=row[0][1],
        revenue=row[1][0],
        count=row[1][1]
    )

In [23]:
from pyspark.sql import types

In [24]:
result_schema = types.StructType([
    types.StructField('hour', types.TimestampType(), True),
    types.StructField('zone', types.IntegerType(), True),
    types.StructField('revenue', types.DoubleType(), True),
    types.StructField('count', types.IntegerType(), True)
])

In [25]:
df_result = rdd \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .reduceByKey(calculate_revenue) \
    .map(unwrap) \
    .toDF(result_schema) 

Traceback (most recent call last):
  File "/workspaces/data-engineering-zoomcamp-homework/05-batch/spark/spark-3.3.2-bin-hadoop3/python/pyspark/serializers.py", line 458, in dumps
    return cloudpickle.dumps(obj, pickle_protocol)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/workspaces/data-engineering-zoomcamp-homework/05-batch/spark/spark-3.3.2-bin-hadoop3/python/pyspark/cloudpickle/cloudpickle_fast.py", line 73, in dumps
    cp.dump(obj)
  File "/workspaces/data-engineering-zoomcamp-homework/05-batch/spark/spark-3.3.2-bin-hadoop3/python/pyspark/cloudpickle/cloudpickle_fast.py", line 602, in dump
    return Pickler.dump(self, obj)
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "/workspaces/data-engineering-zoomcamp-homework/05-batch/spark/spark-3.3.2-bin-hadoop3/python/pyspark/cloudpickle/cloudpickle_fast.py", line 692, in reducer_override
    return self._function_reduce(obj)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/workspaces/data-engineering-zoomcamp-homework/05-ba

PicklingError: Could not serialize object: IndexError: tuple index out of range

In [50]:
df_result.write.parquet('tmp/green-revenue')

# mapPartitions - Gd for large datasets
* input 1 partition
* apply map fn
* output 1 partition

In [55]:
columns = ['VendorID', 'lpep_pickup_datetime', 'PULocationID', 'DOLocationID', 'trip_distance']

duration_rdd = df_green \
    .select(columns) \
    .rdd

In [67]:
import pandas as pd

In [68]:
rows = duration_rdd.take(10)

In [81]:
df = pd.DataFrame(rows, columns=columns)

In [74]:
columns

['VendorID',
 'lpep_pickup_datetime',
 'PULocationID',
 'DOLocationID',
 'trip_distance']

In [76]:
#model = ...

def model_predict(df):
#     y_pred = model.predict(df)
    y_pred = df.trip_distance * 5
    return y_pred

In [98]:
def apply_model_in_batch(rows):
    df = pd.DataFrame(rows, columns=columns)

    # 2026.01 fix
    # Spark does not recognise the Pandas Timestamp, converting it to string will solve the empty issues
    df['lpep_pickup_datetime'] = df['lpep_pickup_datetime'].astype(str) # string conversion
    
    predictions = model_predict(df)
    df['predicted_duration'] = predictions

    for row in df.itertuples():
        yield row

In [102]:
df_predicts = duration_rdd \
    .mapPartitions(apply_model_in_batch)\
    .toDF() \
    .drop('Index')

In [104]:
df_predicts.select('predicted_duration').show()

+------------------+
|predicted_duration|
+------------------+
|             12.95|
|             31.25|
|              14.0|
|             12.75|
|               0.1|
|             11.05|
|11.299999999999999|
|54.349999999999994|
|             15.25|
|             91.75|
|             12.25|
|               3.1|
|               7.5|
|11.899999999999999|
| 78.89999999999999|
|              4.45|
|              23.2|
|              4.85|
|              6.65|
|              15.1|
+------------------+
only showing top 20 rows

